# Los Angeles (South Bay) Urgent Care Location Analysis

In [1]:
!pip install geopy
!pip install folium
import pandas as pd
import numpy as np
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim
import folium # map rendering library
print('Libraries imported.')

Libraries imported.


# IMPORT DATA CSV

In [2]:
# The code was removed by Watson Studio for sharing.

,Neighborhood,Zip,Population,District
0,Inglewood,90009,58104,4
1,HERMOSA BEACH,90254,125328,4
2,MANHATTAN BEACH,90266,221231,4
3,PALOS VERDES PENINSULA,90274,287764,4
4,RANCHO PALOS VERDES,90275,142191,4


# IMPORT LAT/LONG CSV FILE

In [3]:


body = client_f32f5c4920084615b15c4d597e57d07c.get_object(Bucket='applieddatasciencecapstone-donotdelete-pr-3zecmyooy6vd6w',Key='latlong-1.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

lat_long = pd.read_csv(body)
lat_long.head()


,Zipcode,Lat,Long
0,96162,39.30,-120.31
1,96161,39.33,-120.24
2,96160,39.32,-120.18
3,96158,38.92,-119.96
4,96157,38.94,-119.97


# FORMAT AND MERGE

In [4]:

la_data.rename(columns={'Zip': 'Zipcode'}, inplace=True)
la_data.set_index('Zipcode')
lat_long.set_index('Zipcode')
comboLA = pd.merge(la_data, lat_long, on='Zipcode')

# Create neighboarhood and population datafram for later use
comboLAgrp = comboLA.drop(['Zipcode', 'Lat','Long','District'], axis=1)
comboLAgrp = comboLAgrp.groupby('Neighborhood').sum()

# SET INITIAL MAP PARAMETERS AND CREATE MAP

In [5]:

address = 'Lawndale, California'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

In [6]:
# create map of LA
map_la = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, District, Neighborhood in zip(comboLA['Lat'], comboLA['Long'], comboLA['District'], comboLA['Neighborhood']):
    label = '{}, {}'.format(Neighborhood, District)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_la)  
    
map_la

# SET FOURSQUARE PARAMETERS

In [7]:

CLIENT_ID = 'AKCX2GTFV10033CD3WV2BEJMD1OCM2DOP0JWBMPGTNW54G0B' # your Foursquare ID
CLIENT_SECRET = 'UGZADPTBYFFWVG5R31GBEQBFKZTUGC5UMVGSYPXTI2C0GEGK' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [8]:
CATID = '56aa371be4b08b9a8d573526'
LIMIT = 50

# DEFINE FUNCTION TO PROCESS GET REQUESTS

In [9]:


def getNearbyVenues(names, latitudes, longitudes, radius=5000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT,
            CATID)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

# LOOP AND RETURN VALUES


In [10]:

la_venues = getNearbyVenues(names=comboLA['Neighborhood'],
                                   latitudes=comboLA['Lat'],
                                   longitudes=comboLA['Long']
                                  )

Inglewood
HERMOSA BEACH
MANHATTAN BEACH
PALOS VERDES PENINSULA
RANCHO PALOS VERDES
REDONDO BEACH
REDONDO BEACH
TORRANCE
TORRANCE
MARINA DEL REY
PLAYA DEL REY
INGLEWOOD
INGLEWOOD
INGLEWOOD
SANTA MONICA
TORRANCE
HARBOR CITY
WILMINGTON
CARSON
CARSON
HAWTHORNE
LAWNDALE


# CONSOLIDATE NUMBER OF LOCATIONS AND NEIGHBORHOOD


In [11]:

consol_venue = la_venues['Neighborhood'].value_counts()

# CREATE NEW DATAFRAME THAT COMBINES Neighborhood, Count, and Population 


In [12]:

LAfinal = pd.DataFrame({'Neighborhood':consol_venue.index, 'Urgent Care Count':consol_venue.values, 'Population':comboLAgrp.Population}).set_index('Neighborhood')

# Add value for urgent cares per 10,000 residents

In [13]:

LAfinal['UC per 10,000'] = LAfinal['Urgent Care Count']/LAfinal['Population']*10000

# sort values so that the neighborhood with the fewest urgent care locations per 10,000 residents are towards the top

In [14]:
LAfinal.sort_values(by=['UC per 10,000'])

,Urgent Care Count,Population,"UC per 10,000"
Neighborhood,,,
RANCHO PALOS VERDES,3,175631,0.170813
Inglewood,5,287764,0.173753
WILMINGTON,4,172386,0.232037
CARSON,4,142191,0.281312
HAWTHORNE,3,104474,0.287153
HERMOSA BEACH,7,221231,0.316411
PALOS VERDES PENINSULA,4,93632,0.427204
HARBOR CITY,6,109454,0.548175
SANTA MONICA,11,125328,0.877697


# This analysis has concluded that the best neighborhoods in the South Bay for opening an Urgent Care based on competition are Hawthorne and Inglewood